In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210501.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,27/04/2021 17:39:51,Moonstruck (1987),Amarcord (1973),The Bird with the Crystal Plumage (1970),L'armata Brancaleone (1966),PHYSICAL COMEDY,INDIGENOUS STORIES,[Default to Nicolas Cage],PROBLEMATIC FAVES,PET FANCY
1,27/04/2021 17:50:24,Amarcord (1973),The Bird with the Crystal Plumage (1970),L'armata Brancaleone (1966),Moonstruck (1987),PET FANCY,PROBLEMATIC FAVES,PHYSICAL COMEDY,[Default to Nicolas Cage],INDIGENOUS STORIES
2,27/04/2021 17:56:13,Moonstruck (1987),L'armata Brancaleone (1966),The Bird with the Crystal Plumage (1970),Amarcord (1973),INDIGENOUS STORIES,PROBLEMATIC FAVES,[Default to Nicolas Cage],PHYSICAL COMEDY,PET FANCY
3,27/04/2021 18:26:34,Amarcord (1973),The Bird with the Crystal Plumage (1970),L'armata Brancaleone (1966),Moonstruck (1987),PROBLEMATIC FAVES,INDIGENOUS STORIES,PHYSICAL COMEDY,PET FANCY,[Default to Nicolas Cage]
4,27/04/2021 18:31:27,L'armata Brancaleone (1966),Moonstruck (1987),The Bird with the Crystal Plumage (1970),NaN,INDIGENOUS STORIES,PHYSICAL COMEDY,PROBLEMATIC FAVES,[Default to Nicolas Cage],NaN
5,29/04/2021 08:20:14,Amarcord (1973),The Bird with the Crystal Plumage (1970),Moonstruck (1987),L'armata Brancaleone (1966),PROBLEMATIC FAVES,PHYSICAL COMEDY,PET FANCY,INDIGENOUS STORIES,[Default to Nicolas Cage]
6,29/04/2021 12:01:27,The Bird with the Crystal Plumage (1970),Amarcord (1973),L'armata Brancaleone (1966),Moonstruck (1987),PROBLEMATIC FAVES,PHYSICAL COMEDY,INDIGENOUS STORIES,PET FANCY,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Moonstruck (1987)', 'Amarcord (1973)',
       'The Bird with the Crystal Plumage (1970)',
       "L'armata Brancaleone (1966)"], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Moonstruck (1987)': <Candidate('Moonstruck (1987)')>,
 'Amarcord (1973)': <Candidate('Amarcord (1973)')>,
 'The Bird with the Crystal Plumage (1970)': <Candidate('The Bird with the Crystal Plumage (1970)')>,
 "L'armata Brancaleone (1966)": <Candidate('L'armata Brancaleone (1966)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Moonstruck (1987), Amarcord (1973), The Bird with the Crystal Plumage (1970), L'armata Brancaleone (1966))>,
 <Ballot(Amarcord (1973), The Bird with the Crystal Plumage (1970), L'armata Brancaleone (1966), Moonstruck (1987))>,
 <Ballot(Moonstruck (1987), L'armata Brancaleone (1966), The Bird with the Crystal Plumage (1970), Amarcord (1973))>,
 <Ballot(Amarcord (1973), The Bird with the Crystal Plumage (1970), L'armata Brancaleone (1966), Moonstruck (1987))>,
 <Ballot(L'armata Brancaleone (1966), Moonstruck (1987), The Bird with the Crystal Plumage (1970))>,
 <Ballot(Amarcord (1973), The Bird with the Crystal Plumage (1970), Moonstruck (1987), L'armata Brancaleone (1966))>,
 <Ballot(The Bird with the Crystal Plumage (1970), Amarcord (1973), L'armata Brancaleone (1966), Moonstruck (1987))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                                   Votes  Status
----------------------------------------  -------  --------
Amarcord (1973)                                 3  Hopeful
Moonstruck (1987)                               2  Hopeful
The Bird with the Crystal Plumage (1970)        1  Rejected
L'armata Brancaleone (1966)                     1  Rejected

FINAL RESULT
Candidate                                   Votes  Status
----------------------------------------  -------  --------
Amarcord (1973)                                 4  Elected
Moonstruck (1987)                               3  Rejected
The Bird with the Crystal Plumage (1970)        0  Rejected
L'armata Brancaleone (1966)                     0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                    Votes  Status
-------------------------  -------  --------
PROBLEMATIC FAVES                3  Hopeful
INDIGENOUS STORIES               2  Hopeful
PHYSICAL COMEDY                  1  Rejected
PET FANCY                        1  Rejected
[Default to Nicolas Cage]        0  Rejected

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
PROBLEMATIC FAVES                4  Elected
INDIGENOUS STORIES               3  Rejected
PHYSICAL COMEDY                  0  Rejected
PET FANCY                        0  Rejected
[Default to Nicolas Cage]        0  Rejected

